# IMPORTS

In [1]:
from hvac_control.data import load_data, save_img, save_data
from hvac_control.preprocessing import *
from hvac_control.decision_plots import *

#load preprocessed data
data = load_data("gaia_data_1.csv", data_type='processed')

In [2]:
x_columns = ['Day', 'T_ext', 'Solar_irrad', 'T_imp', 
           'BC1_power', 'BC2_power', 'Diff_temp',
           'Hours_sin', 'Hours_cos', 'T_ret'] 

y_column = ['T_ret_in_1h']

# SPLIT

In [4]:
day_for_training = 14
data_for_train = data[data['Day'] != day_for_training]
data_for_test = data[data['Day'] == day_for_training]

train_x = data_for_train[x_columns]
train_y = data_for_train[y_column]

test_x = data_for_test[x_columns]
test_y = data_for_test[y_column]

print('len train', train_x.shape[0])
print('len test', test_x.shape[0])

len train 5264
len test 376


# SCALING

In [5]:
from hvac_control.preprocessing import minmax_scaler_given_parameters, std_scaler_given_parameters

mu_x = train_x.mean(0)
s_x = train_x.std(0)

mu_y = train_y.mean(0)
s_y = train_y.std(0)

max_x = train_x.max(0)
min_x = train_x.min(0)

max_y = train_y.max(0)
min_y = train_y.min(0)

train_x_norm = std_scaler_given_parameters(train_x, mu_x, s_x)
test_x_norm = std_scaler_given_parameters(test_x, mu_x, s_x)
train_y_norm = std_scaler_given_parameters(train_y, mu_y, s_y)
test_y_norm = std_scaler_given_parameters(test_y, mu_y, s_y)

# train_x_norm = minmax_scaler_given_parameters(train_x,  max_val=max_x, min_val=min_x, feature_range=[0,1])
# test_x_norm = minmax_scaler_given_parameters(test_x,  max_val=max_x, min_val=min_x, feature_range=[0,1])
# train_y_norm = minmax_scaler_given_parameters(train_y,  max_val=max_y, min_val=min_y, feature_range=[0,1])
# test_y_norm = minmax_scaler_given_parameters(test_y,  max_val=max_y, min_val=min_y, feature_range=[0,1])

# LSTM MODEL CREATION

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf

win_length=70
batch_size=70
num_features=9

train_x_arr=train_x.to_numpy()
train_y_arr=train_y.to_numpy()
test_x_arr=test_x.to_numpy()
test_y_arr=test_y.to_numpy()

train_generator = TimeseriesGenerator(train_x_arr, train_y_arr, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(test_x_arr, test_y_arr, length=win_length, sampling_rate=1, batch_size=batch_size)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(win_length, num_features), return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.LSTM(64, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1))

# LSTM TRAINING

In [17]:
model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),
              metrics=[tf.metrics.MeanAbsoluteError()])

history = model.fit_generator(train_generator, epochs=200,
                              validation_data=test_generator,
                              shuffle=False,)
                              #callbacks=[early_stopping])

model.evaluate_generator(test_generator, verbose=0)

predictions = model.predict_generator(test_generator)




Epoch 1/175
165/165 [==============================] - 4s 25ms/step - loss: 0.4529
Epoch 2/175
165/165 [==============================] - 4s 24ms/step - loss: 0.4292
Epoch 3/175
165/165 [==============================] - 4s 24ms/step - loss: 0.4205
Epoch 4/175
165/165 [==============================] - 4s 24ms/step - loss: 0.4133
Epoch 5/175
165/165 [==============================] - 4s 25ms/step - loss: 0.4023
Epoch 6/175
165/165 [==============================] - 4s 24ms/step - loss: 0.3931
Epoch 7/175
165/165 [==============================] - 4s 24ms/step - loss: 0.3842
Epoch 8/175
165/165 [==============================] - 4s 25ms/step - loss: 0.3801
Epoch 9/175
165/165 [==============================] - 4s 24ms/step - loss: 0.3745
Epoch 10/175
165/165 [==============================] - 4s 25ms/step - loss: 0.3708
Epoch 11/175
165/165 [==============================] - 4s 24ms/step - loss: 0.3659
Epoch 12/175
165/165 [==============================] - 4s 25ms/step - loss: 0.361

In [ ]:
model.evaluate_generator(test_generator, verbose=0)
predictions = model.predict_generator(test_generator)
predictions.shape[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

time = np.arange(0,len(test_y[:-win_length]))
a=s_y*test_y+mu_y
predicted_data_df = pd.DataFrame()
predicted_data_df['T_ret_in_1h']=pd.Series(predictions.ravel())
b=s_y*predicted_data_df+mu_y
a=a.iloc[:-70]

error_sum = (np.sum(abs(a.to_numpy()-b.to_numpy())))
error_avg = (np.mean(abs(a.to_numpy()-b.to_numpy())))

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(time, a, label='Actual Data', color='blue')
plt.plot(time, b, label='Predicted Data', color='red')
plt.title('Actual Data vs Predicted Data')
plt.xlabel('Time or Observation Number')
plt.ylabel('Data Value')
plt.figtext(.95, 0.8, f"error_sum = {error_sum}") 
plt.figtext(.95, 0.8, f"error_avg = {error_avg}") 
plt.legend()
plt.show()